# Google Analytics Customer Revenue Prediction

## 0. Project Description and Methodology:
The 80/20 rule has proven true for many businesses–only a small percentage of customers produce most of the revenue. As such, marketing teams are challenged to make appropriate investments in promotional strategies.

In this project, we'll analyze a Google Merchandise Store (also known as GStore, where Google swag is sold) customer dataset to predict revenue per customer. Hopefully, the outcome will be more actionable operational changes and a better use of marketing budgets for those companies who choose to use data analysis on top of GA data.

<a href="https://shop.googlemerchandisestore.com/">
<img src="https://shop.googlemerchandisestore.com/store/20160512512/assets/items/images/GGOEGBJQ125099.jpg" style="width:300px;height:300px;" alt="Google Tote bage">
</a>

In this project, we'll utilize the process of we learned in class as a guideline to go through the analysis:

* [**<b>1. Business understanding</b>**](#1)
    - In this part, we'll conduct some peripheral research to get a more comprehensive understanding of the whole business model. 
* [**<b>2. Data understanding</b>**](#2)
    - Dig into the available dataset and try to comprehent the business and extract the data which is needed. Handling missing values.
* [**<b>3. Data preparation</b>**](#3)
    - Understand the dependencies between attributes. Analyzing the target variables. Transforming data formats to standard data format.
* [**<b>4. Modeling</b>**](#4)
    - Selecting more accurate classfier or regression engine based on the charactristic any of them have.
    - Train models that are pertinent to the dataset.

# 1. Business Understanding: <a id="1"></a>

Google Merchandise Store is official store of Google that contains merchandises that are produced by the company. The company is currently looking for new and effective ways of marketing this website to make customers aware about their merchandises and connect with them appropriately. For this, a Digital Media Marketing Plan must be developed for the company to achieve this agenda and to effectively develop integrated marketing communication strategy for the company.

# 2. Data understanding<a id="2"></a>

* **<b>A. IMPORTS</b>**

Importing necessary packages and libraries.

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import timedelta, date
import seaborn as sns
import matplotlib.cm as CM
import collections
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor , plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, train_test_split
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

* **B. READING DATA**

First of all, get a sense of the whole dataset by reading the description.

Reading data and caughting a glimpse of what data it is.

In [ ]:
path = "../input/ga-customer-revenue-prediction/train.csv"
train_data = pd.read_csv(path)
train_data.head()

By using the shape attribute of the pandas dataframe, we could see the train_data has a dimension of 903653 * 12.

In [ ]:
print (train_data.shape)
print("\n")
print(list(train_data.columns.values))

* **C. FEATURES DESCRIPTION**

Referring back to Data description to further understanding the features.

*     channelGrouping - The channel via which the user came to the Store.
*     date - The date on which the user visited the Store.
*     device - The specifications for the device used to access the Store.
*     fullVisitorId- A unique identifier for each user of the Google Merchandise Store.
*     geoNetwork - This section contains information about the geography of the user.
*     sessionId - A unique identifier for this visit to the store.
*     socialEngagementType - Engagement type, either "Socially Engaged" or "Not Socially Engaged".
*     totals - This section contains aggregate values across the session.
*     trafficSource - This section contains information about the Traffic Source from which the session originated.
*     visitId - An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.
*     visitNumber - The session number for this user. If this is the first session, then this is set to 1.
*     visitStartTime - The timestamp (expressed as POSIX time).

* **D. INDIVIDUAL FEATURE UNDERSTANDING**

In this section, we're going to analyze these features separately to get a deeper understanding.

**- D1. channelGrouping**

As decribed in the decription, the channelGrouping feature indicates the different channels which lead the customers to visit the GStore, it is quite an important feature to let us understand more precisely where the customers are from.

In [ ]:
tmp = train_data.channelGrouping.value_counts()
labels = tmp.index
sizes = tmp.values

fig1, ax1 = plt.subplots(figsize=(8,8))
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, textprops=dict(color="w"))
ax1.legend(labels, loc="center right",bbox_to_anchor=(0.8, 0, 0.5, 1))

plt.show()

In [ ]:
snstemp = pd.DataFrame(train_data.channelGrouping.value_counts())
sns.barplot(y = snstemp.index, x = snstemp.iloc[:,0])

We can see a general distribution of this feature.

**- D2. date & visitStartTime **

The two varialbes, date and visitStartTime, are related to time and can be used in time dependent analyzes specially TimeSeries.

First of all, we can use the first row to have a glance of the variables.

In [ ]:
print("date :{}, visitStartTime:{}".format(train_data.head(1).date[0],train_data.head(1).visitStartTime[0]))

Thus, we could see that date is stored in String and should be converted to pandas datetime format. While, visitStartTime is stored in epoch unix format and should be converted to pandas datetime format.

In [ ]:
train_data["date"] = pd.to_datetime(train_data["date"],format="%Y%m%d")
train_data["visitStartTime"] = pd.to_datetime(train_data["visitStartTime"],unit='s')

In [ ]:
train_data.head()[["date","visitStartTime"]]

Through such conversion, we could get a more intuitive understanding of the visit time related to every row.

** - D3. device **

The device variable is stored in json format. We need to extract its fields and analyze them by utilizing json library to deserializing json values into a pandas dataframe.

In [ ]:
tmp_device_df = pd.DataFrame(train_data.device.apply(json.loads).tolist())

In [ ]:
# replace the JSON's "not available in demo dataset" and "(not set)" to NULL
tmp_device_df = tmp_device_df.replace(["not available in demo dataset","(not set)"], pd.NaT)
tmp_device_df.head()

In [ ]:
tmp_device_df.nunique()

Keys existed in device attribute are listed above and we could use the nunique function to list out how many unique values are there in one key. We could see that, except four keys, all the other keys are constant with only one unique value. Therefore, we should ignore the features which are not usefull in rest of the process. If feature is misrelated, or it contains lot of "NaN" values it should be discarded. We select the ***["browser","operatingSystem","deviceCategory","isMobile"]*** for doing the analyzing. The rest of the device features are ignored and will be removed.

We can redistribute the dataframe and contain only the useful columns.

In [ ]:
tmp_device_df = pd.DataFrame(train_data.device.apply(json.loads).tolist())[["browser","operatingSystem","deviceCategory","isMobile"]]
tmp_device_df = tmp_device_df.replace(["not available in demo dataset","(not set)"], pd.NaT)

In [ ]:
fig, axes = plt.subplots(2,2,figsize=(16,16))
tmp_device_df["isMobile"].value_counts().plot(kind="barh",ax=axes[0][0],legend="isMobile",color='tan').invert_yaxis()
tmp_device_df["browser"].value_counts().head(10).plot(kind="barh",ax=axes[0][1],legend="browser",color='teal').invert_yaxis()
tmp_device_df["deviceCategory"].value_counts().head(10).plot(kind="barh",ax=axes[1][0],legend="deviceCategory",color='green').invert_yaxis()
tmp_device_df["operatingSystem"].value_counts().head(10).plot(kind="barh",ax=axes[1][1],legend="operatingSystem",color='c').invert_yaxis()

** - D4. geoNetwork **

The geoNetwork variable is stored in json format. We can use the same way to decompose the variable and pick the useful variables.

In [ ]:
tmp_geo_df = pd.DataFrame(train_data.geoNetwork.apply(json.loads).tolist())

In [ ]:
# replace the JSON's "not available in demo dataset" and "(not set)" to NULL
tmp_geo_df = tmp_geo_df.replace(["not available in demo dataset","(not set)"], pd.NaT)
tmp_geo_df.head()

In [ ]:
tmp_geo_df.nunique()

In [ ]:
tmp_geo_df.isna().sum()

From the above analysis, we could conclude that: 1. "cityId", "latitude", "longitude" and "networkLocation" contain no value; 2. "networkDomain", "region" have too many null value. So we'll just drop these variables and keep "continent", "subContinent", "country", "city", "metro".

In [ ]:
tmp_geo_df = pd.DataFrame(train_data.geoNetwork.apply(json.loads).tolist())[["continent", "subContinent", "country", "city", "metro"]]
tmp_geo_df = tmp_geo_df.replace(["not available in demo dataset","(not set)"], pd.NaT)  
tmp_geo_df.head()

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(15,15))
tmp_geo_df["continent"].value_counts().plot(kind="bar",ax=axes[0][0],title="Global Distributions",rot=0,color="c")
tmp_geo_df[tmp_geo_df["continent"] == "Americas"]["subContinent"].value_counts().plot(kind="bar",ax=axes[1][0], title="America Distributions",rot=0,color="tan")
tmp_geo_df[tmp_geo_df["continent"] == "Asia"]["subContinent"].value_counts().plot(kind="bar",ax=axes[0][1], title="Asia Distributions",rot=0,color="r")
tmp_geo_df[tmp_geo_df["continent"] == "Europe"]["subContinent"].value_counts().plot(kind="bar",ax=axes[1][1],  title="Europe Distributions",rot=0,color="green")
tmp_geo_df[tmp_geo_df["continent"] == "Oceania"]["subContinent"].value_counts().plot(kind="bar",ax = axes[2][0], title="Oceania Distributions",rot=0,color="teal")
tmp_geo_df[tmp_geo_df["continent"] == "Africa"]["subContinent"].value_counts().plot(kind="bar" , ax=axes[2][1], title="Africa Distributions",rot=0,color="silver")

** - D5. socialEngagementType **

From the summary, we could see that there's only one unique data, thus, we should drop this column.

In [ ]:
train_data["socialEngagementType"].describe()

** - D6. totals **

The totals variable is stored in json format. We could use the same method to clean the data.

First of all, we need to understand the meaning of these features by reading the <a href= "https://support.google.com/analytics/answer/2731565?hl=en"> Google Analytics definition</a>:

1. **hits** : hit is an interaction that results in data being sent to Analytics. Common hit types include page tracking hits, event tracking hits, and ecommerce hits.
1. **bounces**: Google Analytics defines a bounce as a session that has a single interaction hit. Interaction hits are pageviews and other interaction events you may have configured on your page. If you have no other interaction events configured, a bounce is simply a session that has a single pageview. The bounce rate is simply the percent of all sessions that are bounces. If out of 100 sessions, 75 of them had a single pageview and no other interaction hit, then the bounce rate is 75%.
1. **pageview** : pageview is an instance of a page being loaded (or reloaded) in a browser. Pageviews is a metric defined as the total number of pages viewed.
1. **newVisits** : newVisits is a binary feature to mark if the user is a new visitor.
1. **transactionRevenue** : The actual revenue generated in the paticular session. In this dataset, the data is in USD * 1000000 (to maintain precision).

In [ ]:
tmp_totals_df = pd.DataFrame(train_data.totals.apply(json.loads).tolist())

In [ ]:
# replace the JSON's "not available in demo dataset" and "(not set)" to NULL
tmp_totals_df = tmp_totals_df.replace(["not available in demo dataset","(not set)"], pd.NaT)

In [ ]:
tmp_totals_df.describe()

From the above analysis, we could conclude that:"visits", "bounces", and "newVisits" contain only one unique value. But by digging deeper in these three parameters, we could see that these features are binary features, so that NULL means 0 in these features. So we could impute these NULL data with 0.

In [ ]:
tmp_totals_df = tmp_totals_df.fillna(0)
tmp_totals_df.describe()

So that we could see only one feature "visits" still contain only one value, we should drop this feature and use the other five features.

In [ ]:
tmp_totals_df = tmp_totals_df.drop("visits", axis = 1)

Since transcationRevenue is the value we're going to predict, we're going to dig a little deeper to get a more comprehensive understanding.

In [ ]:
# Printing some statistics of our data
# Convert the data type from string to float 
tmp_totals_df = tmp_totals_df.astype(float)

print("Transaction Revenue Min Value: ", 
      tmp_totals_df[tmp_totals_df['transactionRevenue'] > 0]["transactionRevenue"].min()) # printing the min value
print("Transaction Revenue Mean Value: ", 
      tmp_totals_df[tmp_totals_df['transactionRevenue'] > 0]["transactionRevenue"].mean()) # mean value
print("Transaction Revenue Median Value: ", 
      tmp_totals_df[tmp_totals_df['transactionRevenue'] > 0]["transactionRevenue"].median()) # median value
print("Transaction Revenue Max Value: ", 
      tmp_totals_df[tmp_totals_df['transactionRevenue'] > 0]["transactionRevenue"].max()) # the max value

In [ ]:
plt.figure(figsize=(14,5))

plt.subplot(1,2,1)
tmp = [ len(tmp_totals_df[tmp_totals_df['transactionRevenue'] == 0]),len(tmp_totals_df[tmp_totals_df['transactionRevenue'] > 0])]
lebels = ["Sessions w/o revenue","Sessions w/ revenue"]

plt.pie(tmp, autopct='%1.1f%%', startangle=90, textprops=dict(color="w",fontsize= 14))
plt.legend(lebels,loc="up right",bbox_to_anchor=(0.8, 0, 0.5, 1))

# seting the distribuition of our data and normalizing using np.log on values highest than 0 and + 
# also, we will set the number of bins and if we want or not kde on our histogram
plt.subplot(1,2,2)
ax = sns.distplot(np.log(tmp_totals_df[tmp_totals_df['transactionRevenue'] > 0]["transactionRevenue"]), bins=60, kde=True)
ax.set_xlabel('Transaction RevenueLog', fontsize=15) #seting the xlabel and size of font
ax.set_ylabel('Distribuition', fontsize=15) #seting the ylabel and size of font
ax.set_title("Distribuition of Revenue Log", fontsize=20) #seting the title and size of font

plt.show()

** - D7. trafficSource **

trafficSource attribute can be used to resolve the question of what's the most conventional manner for visitor who visit to the website and do their shopping. Like a previous Json elements existed in the dataset, this attribute is also Json file. so, we use the similar way to deserialize it. We have select "keyword", "source" and the "medium" as a features which can bring more useful infromation.

In [ ]:
traffic_source_df = pd.DataFrame(train_data.trafficSource.apply(json.loads).tolist())[["keyword","medium" , "source"]]
traffic_source_df.head()

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(15,10))
traffic_source_df["medium"].value_counts().plot(kind="barh",ax = axes[0],title="Medium",color="tan").invert_yaxis()
traffic_source_df["source"].value_counts().head(10).plot(kind="barh",ax=axes[1],title="source",color="teal").invert_yaxis()

As it is completely obvious in source diagram, google is the most repetitive source. It would be interesting if we replace all google subdomains with exact 'google' and do the same analyze again.

In [ ]:
traffic_source_df.loc[traffic_source_df["source"].str.contains("google") ,"source"] = "google"
fig,axes = plt.subplots(1,1,figsize=(8,8))
traffic_source_df["source"].value_counts().head(15).plot(kind="barh",ax=axes,title="source",color="teal").invert_yaxis()

Google dependent redirects are more than twice the youtube sources. Combination of this feature with revenue and visits may have important result. We will do it in next step (when we are analyzing feature correlations). Now let's move on keywords feature. A glance to keyword featre represnets lot of missing values '(not provided)'. Drawing a bar chart for both of them.

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(16,8))
traffic_source_df["keyword"].value_counts().head(5).plot(kind="barh",ax=axes[0], title="keywords (total)",color="orange").invert_yaxis()
traffic_source_df[traffic_source_df["keyword"] != "(not provided)"]["keyword"].value_counts().head(10).plot(kind="barh",ax=axes[1],title="keywords (dropping NA)",color="c").invert_yaxis()

** -D8. visitId & sessionId**

These two features are acting the same function to index the dataset. We could ignore them since we're going to predict the customer's behavior instead of what's the revenue will be in a single session.

** -D9. visitNumber **

The feature indicates how many times a specific user visits the store. By intuition, this feature has a potential to be an important factor in regression progress.

In [ ]:
train_data.visitNumber.describe()

We could see that 75% of the users only visit the store for one time, so we could say that maybe majority of the visitors are lured to the store by marketing campaign and are not our target customers. We could further understand the situation to see the 90% of the users' behaviors. We can do the further analysis to see the correlation between the total visits and the revenue.

In [ ]:
print("90 percent of sessions have visitNumber lower than {} times.".format(np.percentile(list(train_data.visitNumber),90)))

** -D10. fullVisitorId **

As decribed, this feature is a unique identifier for each user of the Google Merchandise Store. We could get that in our dataset, there're 742735 different users.

In [ ]:
train_data.fullVisitorId.nunique()

# 3. Data preparation <a id="3"></a>

With the understanding of the features in the dataset, in this section, we're going dig deeper to construct a more comprehensive view by combining the features together to see the interrelationship between the features.

* A. transactionRevenue and other features

In order to find out the relationship between transaction revenue with other features, we could first write a function to plot out the relationship between different columns and the available transaction revenue.
First we need to merge all the useful features we talked above and get a cleaned dataset.

In [ ]:
# Merge all the useful features back together after the cleaning procedures above.
train_data_cleaned = train_data[["fullVisitorId", "channelGrouping", "date", "visitStartTime","visitNumber"]]
train_data_cleaned = train_data_cleaned.merge(tmp_device_df,left_index=True, right_index=True)
train_data_cleaned = train_data_cleaned.merge(tmp_geo_df,left_index=True, right_index=True)
train_data_cleaned = train_data_cleaned.merge(tmp_totals_df,left_index=True, right_index=True)
train_data_cleaned = train_data_cleaned.merge(traffic_source_df,left_index=True, right_index=True)
train_data_cleaned.head(1)

Then we could subset the dataset, and put all the rows with revenue together.

In [ ]:
train_data_rev = train_data_cleaned[train_data_cleaned['transactionRevenue'] > 0]
train_data_rev.head(1)

Then we can write a function to plot the relationship between the overall dataset and the rows with revenue.

In [ ]:
def plotCategoryRateBar(a, b, colName, topN=np.nan):
    if topN == topN:
        vals = b[colName].value_counts()[:topN]
        subA = a.loc[a[colName].isin(vals.index.values), colName]
        df = pd.DataFrame({'% in Rows with Revenue':subA.value_counts() / len(a) *100, '% in Overall Dataset':vals / len(b)*100})
    else:
        df = pd.DataFrame({'% in Rows with Revenue':a[colName].value_counts() / len(a)*100, '% in Overall Dataset':b[colName].value_counts() / len(b)*100})
    #return the barh plot
    df.sort_values('% in Rows with Revenue').plot.barh(colormap='bwr', title = colName +" and Revenue relationship")

We can also write a function to plot the relationship between the total/mean/median revenue.

Through this analysis, we could understand the total revenue and the unit economic from a certain group of customers.

In [ ]:
def plotCategoryAvgBar(a, colName,topN=np.nan):
    df = pd.DataFrame()
    
    for item in a[colName].unique():
        mean = np.mean(a[a[colName] == item].transactionRevenue)/100000
        median = np.median(a[a[colName] == item].transactionRevenue)/100000
        total = np.sum(a[a[colName] == item].transactionRevenue)/100000
        temp_df = pd.DataFrame([[mean,median,total]],index = [item],columns=["mean","median","total"])
        df = pd.concat([df,temp_df])
    
    if topN == topN:
        fig,axes=plt.subplots(1,2,figsize=(14,5))
        df.loc[:,"total"].sort_values().tail(topN).plot.barh(ax=axes[0],color="b",title = colName +" and total revenue relationship")
        df.loc[:,["mean","median"]].sort_values(by = "median").tail(topN).plot.barh(ax=axes[1],colormap='PiYG', title = colName +" and Revenue mean/median relationship")
    else:
        fig,axes=plt.subplots(1,2,figsize=(14,5))
        df.loc[:,"total"].sort_values().plot.barh(ax=axes[0],color="b",title = colName +" and total revenue relationship")
        df.loc[:,["mean","median"]].sort_values(by = "median").plot.barh(ax=axes[1],colormap='PiYG', title = colName +" and Revenue mean/median relationship")

A1. Channel Grouping

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "channelGrouping")

In [ ]:
plotCategoryAvgBar(train_data_rev, "channelGrouping",5)

A2. Device

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "browser",5)

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "operatingSystem",5)

In [ ]:
plotCategoryAvgBar(train_data_rev, "operatingSystem",5)

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "deviceCategory",5)

In [ ]:
plotCategoryAvgBar(train_data_rev, "deviceCategory",5)

A3. geoNetwork

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "continent",5)

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "metro",5)

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "city",5)

A4. trafficSource

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "medium",5)

In [ ]:
plotCategoryRateBar(train_data_rev, train_data_cleaned, "source",5)

* B. Churn & Conversion

> The ChurnRate is The percentage rate at which customers stop subscribing to a service. Churn rate period can be various from day to a year correspoding to business type. In this section, we will compute and visualize the monthly churn rate. Lets do more investigation on features date and fullVisitorId for more detail mining.

In [ ]:
# list how many days of observations in our dataset
date_list = np.sort(list(set(list(train_data["date"]))))
"first_day:'{}' and last_day:'{}' and toal number of data we have is: '{}' days.".format(date_list[0], date_list[-1],len(set(list(train_data["date"]))))

We could see that we have 366 days (12 month = 1 year) from August 2016 to August 2017 data for churn rate calculations. The best period for churn maybe is the monthly churn rate.

We'll define new empty dataframe and do this calculations on it and copy the our requirements on it. For monthly churn rate calculations, we need to check which users visited have visited the website monthly. This information is located in fullVisitorId. We will copy it to new df.

In [ ]:
tmp_churn_df = pd.DataFrame()
tmp_churn_df["date"] = train_data["date"]
tmp_churn_df["year"] = pd.DatetimeIndex(tmp_churn_df["date"]).year
tmp_churn_df["month"] =pd.DatetimeIndex(tmp_churn_df["date"]).month
tmp_churn_df["fullVisitoId"] = train_data["fullVisitorId"]
tmp_churn_df.head()

In [ ]:
"distinct users who visited the website on 2016-08 are:'{}'persons".format(len(set(tmp_churn_df[(tmp_churn_df.year == 2016) & (tmp_churn_df.month == 8) ]["fullVisitoId"])))

In [ ]:
# so that we could use the same method to save the interval visitors in different months
target_intervals_list = [(2016,8),(2016,9),(2016,10),(2016,11),(2016,12),(2017,1),(2017,2),(2017,3),(2017,4),(2017,5),(2017,6),(2017,7)]
intervals_visitors = []
for tmp_tuple in target_intervals_list: 
    intervals_visitors.append(tmp_churn_df[(tmp_churn_df.year == tmp_tuple[0]) & (tmp_churn_df.month == tmp_tuple[1]) ]["fullVisitoId"])

In [ ]:
tmp_matrix = np.zeros((11,11))

for i in range(0,11):
    k = False
    tmp_set = []
    for j in range(i,11): 
        if k:
            tmp_set = tmp_set & set(intervals_visitors[j])
        else:
            tmp_set = set(intervals_visitors[i]) & set(intervals_visitors[j])
        tmp_matrix[i][j] = len(list(tmp_set))
        k = True

In [ ]:
xticklabels = ["interval 1","interval 2","interval 3","interval 4","interval 5","interval 6","interval 7","interval 8",
              "interval 9","interval 10","interval 11"]
yticklabels = [(2016,8),(2016,9),(2016,10),(2016,11),(2016,12),(2017,1),(2017,2),(2017,3),(2017,4),(2017,5),(2017,6),(2017,7)]
fig, ax = plt.subplots(figsize=(11,11))
ax = sns.heatmap(np.array(tmp_matrix,dtype=int), annot=True, cmap="coolwarm",xticklabels=xticklabels,fmt="d",yticklabels=yticklabels)
ax.set_title("Churn-rate heatmap")
ax.set_xlabel("intervals")
ax.set_ylabel("months")

We could see that our monthly churn rate is extremely high. We need to further analyze the customer behaviors behind the high churn rate.

# 4. Modeling <a id="4"></a>

**Model 1. Predict if there's transaction(The possibility of the transaction)**

In [ ]:
train_data_cleaned = train_data_cleaned.drop(['fullVisitorId','date','visitStartTime'],axis=1)

In [ ]:
train_data_binaryprediction= train_data_cleaned
train_data_binaryprediction["ifrevenue"] = 0
for i in range(len(train_data_binaryprediction.transactionRevenue)):
    if train_data_binaryprediction.transactionRevenue[i] != 0: train_data_binaryprediction["ifrevenue"][i] = 1

train_data_binaryprediction = train_data_binaryprediction.drop("transactionRevenue", axis = 1)
train_data_binaryprediction.head()

In [ ]:
df_train_model1, df_test_model1 = train_test_split(train_data_binaryprediction, test_size=0.2, random_state=42)

In [ ]:
categorical_features = ['channelGrouping', 'browser', 'operatingSystem', 'deviceCategory', 'isMobile',
                        'continent', 'subContinent', 'country', 'city','metro', 'keyword', 'medium', 'source']

numerical_features = ['visitNumber', 'newVisits', 'bounces', 'pageviews', 'hits']

for column_iter in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train_model1[column_iter].values.astype('str')) + list(df_test_model1[column_iter].values.astype('str')))
    df_train_model1[column_iter] = lbl.transform(list(df_train_model1[column_iter].values.astype('str')))
    df_test_model1[column_iter] = lbl.transform(list(df_test_model1[column_iter].values.astype('str')))

for column_iter in numerical_features:
    df_train_model1[column_iter] = df_train_model1[column_iter].astype(np.float)
    df_test_model1[column_iter] = df_test_model1[column_iter].astype(np.float)

We will compare models on the validation set using the **log loss** metric. As a reminder, log loss is used to measure the performance of models whose output is a probability prediction between 0 and 1. It is defined as follows:

$$LogLoss = -\frac{1}{n} \sum^{n}_{i=1}[y_{i}log(\hat{p_{i}}) + (1 - y_{i})log(1-\hat{p_{i}})]$$

where

+ $n$ is the number of observations (*i.e.* the number of cars) in the data set
+ $y_{i}$ is the observed realization of car $i$; it equals 1 if the car has a major defect and 0 if not
+ $\hat{p_{i}}$ is the predicted probability of car $i$ having a major defect according to the model[](http://)

In [ ]:
params_model1 = {
    "objective": "binary",
    "metric": "binary_logloss",
    "num_leaves": 30,
    "min_child_samples": 100,
    "learning_rate": 0.1,
    "bagging_fraction": 0.7,
    "feature_fraction": 0.5,
    "bagging_frequency": 5,
    "bagging_seed": 2018,
    "verbosity": -1
}
lgb_train_model1 = lgb.Dataset(df_train_model1.loc[:,df_train_model1.columns != "ifrevenue"], np.log1p(df_train_model1.loc[:,"ifrevenue"]))
lgb_eval_model1 = lgb.Dataset(df_test_model1.loc[:,df_test_model1.columns != "ifrevenue"], np.log1p(df_test_model1.loc[:,"ifrevenue"]), reference=lgb_train_model1)
gbm_model1 = lgb.train(params_model1, lgb_train_model1, num_boost_round=2000, valid_sets=[lgb_eval_model1], early_stopping_rounds=100,verbose_eval=100)

We could know that since we have only 1.5% of data with revenue, it's quite an unbalanced dataset. So our log loss model should have a lower dumb-LogLoss score. We can refer to the dum-LogLoss score <a href="https://medium.com/@fzammito/whats-considered-a-good-log-loss-in-machine-learning-a529d400632d">here</a>, and we could see that our score is significantly lower than the dumb-logloss score, which makes our prediction valid.
<img src="https://miro.medium.com/max/1400/0*eBPkZ24YYGdCSu58.png">

In [ ]:
lgb.plot_importance(gbm_model1,grid=False,height=0.6)

**Model 2. Predict log transactionRevenue**

In [ ]:
df_train = train_data_cleaned
df_train = df_train.drop("ifrevenue",axis = 1)

In [ ]:
df_train, df_test = train_test_split(df_train, test_size=0.2, random_state=42)

df_train["transactionRevenue"] = df_train["transactionRevenue"].astype(np.float)
df_test["transactionRevenue"] = df_test["transactionRevenue"].astype(np.float)
print("We have these columns for our regression problems:\n{}".format(df_train.columns))

In [ ]:
df_train.head(1)

In [ ]:
categorical_features = ['channelGrouping', 'browser', 'operatingSystem', 'deviceCategory', 'isMobile',
                        'continent', 'subContinent', 'country', 'city','metro', 'keyword', 'medium', 'source']

numerical_features = ['visitNumber', 'newVisits', 'bounces', 'pageviews', 'hits']

for column_iter in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(df_train[column_iter].values.astype('str')) + list(df_test[column_iter].values.astype('str')))
    df_train[column_iter] = lbl.transform(list(df_train[column_iter].values.astype('str')))
    df_test[column_iter] = lbl.transform(list(df_test[column_iter].values.astype('str')))

for column_iter in numerical_features:
    df_train[column_iter] = df_train[column_iter].astype(np.float)
    df_test[column_iter] = df_test[column_iter].astype(np.float)

In [ ]:
params = {
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 30,
    "min_child_samples": 100,
    "learning_rate": 0.1,
    "bagging_fraction": 0.7,
    "feature_fraction": 0.5,
    "bagging_frequency": 5,
    "bagging_seed": 2018,
    "verbosity": -1
}
lgb_train = lgb.Dataset(df_train.loc[:,df_train.columns != "transactionRevenue"], np.log1p(df_train.loc[:,"transactionRevenue"]))
lgb_eval = lgb.Dataset(df_test.loc[:,df_test.columns != "transactionRevenue"], np.log1p(df_test.loc[:,"transactionRevenue"]), reference=lgb_train)
gbm = lgb.train(params, lgb_train, num_boost_round=2000, valid_sets=[lgb_eval], early_stopping_rounds=100,verbose_eval=100)

In [ ]:
lgb.plot_importance(gbm,grid=False,height=0.6)